# Importing an `snnTorch` SNN model to LAVA

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/thesis-lava/src/nir" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/nir"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/nir
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/nir


In [2]:
import numpy as np
import nir
import matplotlib.pyplot as plt

nir_network = nir.read("nir_model_cuba.nir")

In [3]:
# Print the network summary
print(f"NIR Network Info: Nº Nodes: {len(nir_network.nodes)} | Nº Edges: {len(nir_network.edges)}")

print(nir_network.nodes["2"])
print(nir_network.edges)


NIR Network Info: Nº Nodes: 6 | Nº Edges: 5
Linear(weight=array([[ 0.03017123,  0.03300593, -0.01486759, ..., -0.04290418,
         0.02527831,  0.03399573],
       [-0.03412941,  0.01275251, -0.00741993, ..., -0.02844698,
        -0.01157845,  0.00848772],
       [ 0.01515196,  0.00230097, -0.02127903, ..., -0.02738289,
         0.02257233, -0.04216863],
       ...,
       [ 0.03463043,  0.00354015, -0.0397058 , ...,  0.01822981,
         0.03574692, -0.00918025],
       [-0.0277856 ,  0.02493642, -0.0305982 , ..., -0.01540129,
         0.03539132, -0.0418357 ],
       [ 0.02076504,  0.00384213, -0.03294706, ...,  0.02236753,
         0.01417992, -0.04224594]], shape=(10, 500), dtype=float32))
[('input', '0'), ('0', '1'), ('1', '2'), ('3', 'output'), ('2', '3')]


In [4]:

MANUAL_CREATION = True

if MANUAL_CREATION:
    # TODO: This is not the original code. I changed it because it had errors..
    """ ng = nir.NIRGraph(
        nodes={
            "input": nir.Input(input_type=np.array([3])),
            # "affine": nir.Affine(weight=np.array([[8, 2, 10], [14, 3, 14]]).T, bias=np.array([1, 2])),
            "lif": nir.CubaLIF(
                # tau=np.array([1] * 2),
                r=np.array([1 / 1e-4] * 2),
                tau_syn=np.array([1] * 2),
                tau_mem=np.array([1] * 2),
                v_leak=np.array([0] * 2),
                v_threshold=np.array([1] * 2),
            ),
            "out": nir.Output(np.array([3])),
        },
        edges=[
            # ("input", "affine"),
            # ("affine", "lif"),
            ("input", "lif"),
            ("lif", "out")
        ],
    ) """
    ng = nir.NIRGraph(
        nodes={
            "input": nir.Input(input_type=np.array([3])),
            "affine": nir.Affine(weight=np.array(
                [[8, 2, 10], [14, 3, 14]]).T, bias=np.array([0, 0])),
            "lif": nir.LIF(
                tau=np.array([1] * 2),
                r=np.array([1] * 2),
                v_leak=np.array([0] * 2),
                v_threshold=np.array([1] * 2),
            ),
            "output": nir.Output(np.array([3])),
        },
        edges=[(0, 1), (1, 2), (2, 3)],
    )

In [5]:
from nir_to_lava_edit import ImportConfig, LavaLibrary, import_from_nir
# from nir_to_lava import ImportConfig, LavaLibrary, import_from_nir

config = ImportConfig(
    dt=1e-4, fixed_pt=False, on_chip=False, library_preference=LavaLibrary.Lava,
)

In [6]:
nir2lava_network = import_from_nir(
    nir_network,  # ng, # 
    config
)

AssertionError: Shapes (500,) and (1,) are incompatible.

In [ ]:
print(nir2lava_network)

I was able to make it run without errors! However, it seems like the code to convert a NIR network to a LAVA network is not complete.

## Import an HDF5 Network into LAVA

In [ ]:
# Import HDF5 network as Lava Processes
from lava.lib.dl.netx import hdf5

# Import the network
net = hdf5.Network(net_config="nir_model_cuba.nir")